In [25]:
# get a new data set concerning four weekdays - June 4, 12, 20, and 24
import pandas as pd
df04 = pd.read_excel('영업소간교통량240604.xlsx')

In [27]:
df12 = pd.read_excel('영업소간교통량240612.xlsx')
df20 = pd.read_excel('영업소간교통량240620.xlsx')
df24 = pd.read_excel('영업소간교통량240624.xlsx')

In [28]:
# extract columns of interest
# pdf stands for Partial dataFrame
pdf04 = df04[['출발영업소코드','도착영업소코드','출발영업소명','도착영업소명','도착지방향총교통량','출발지방향총교통량']]
pdf12 = df12[['출발영업소코드','도착영업소코드','출발영업소명','도착영업소명','도착지방향총교통량','출발지방향총교통량']]
pdf20 = df20[['출발영업소코드','도착영업소코드','출발영업소명','도착영업소명','도착지방향총교통량','출발지방향총교통량']]
pdf24 = df24[['출발영업소코드','도착영업소코드','출발영업소명','도착영업소명','도착지방향총교통량','출발지방향총교통량']]

In [29]:
# sort pdfs by columns '출발영업소코드' and'도착영업소코드'
pdf04 = pdf04.sort_values(by=['출발영업소코드','도착영업소코드'], ascending=[True,True], ignore_index=True)
pdf12 = pdf12.sort_values(by=['출발영업소코드','도착영업소코드'], ascending=[True,True], ignore_index=True)
pdf20 = pdf20.sort_values(by=['출발영업소코드','도착영업소코드'], ascending=[True,True], ignore_index=True)
pdf24 = pdf24.sort_values(by=['출발영업소코드','도착영업소코드'], ascending=[True,True], ignore_index=True)

In [30]:
pdf04 # rudundant rows are found at its tail

,출발영업소코드,도착영업소코드,출발영업소명,도착영업소명,도착지방향총교통량,출발지방향총교통량
0,101,101,서울,서울,75,75
1,101,102,서울,동수원,3994,5521
2,101,103,서울,수원신갈,18728,18228
3,101,104,서울,지곡,14,7
4,101,105,서울,기흥,9895,9846
...,...,...,...,...,...,...
189220,9999,996,전체영업소,연계할인,0,0
189221,9999,997,전체영업소,최장*3,0,0
189222,9999,998,전체영업소,최장,0,0
189223,9999,999,전체영업소,최장*10,0,0


In [ ]:
# truncate rows with invalide station codes by boolean indexing
# 987 is the maximum code according to the list obtained in the first step(fetchOpentAPI_1)
# tdf stands for truncated dataFrame
codeLimit = 987
bMask = (pdf04['출발영업소코드']>codeLimit) | (pdf04['도착영업소코드']>codeLimit)
tdf04 = pdf04[~bMask]
print("tdf01 :\n", tdf04)

# do the same to the rest
# as all are thought to have the same rows, use the bMask01 to the rest as well
tdf12 = pdf12[~bMask]
tdf20 = pdf20[~bMask]
tdf24 = pdf24[~bMask]

# check their tails
print("tdf04 :\n", tdf12.tail())
print("tdf16 :\n", tdf20.tail())
print("tdf20 :\n", tdf24.tail())

In [ ]:
# make copies of tdfs and rename traffic columns
mdf04 = tdf04.copy()
mdf04 = mdf04.rename(columns={'도착지방향총교통량':'도착지방향총교통량04','출발지방향총교통량':'출발지방향총교통량04'})

mdf12 = tdf12.copy()
mdf12 = mdf12.rename(columns={'도착지방향총교통량':'도착지방향총교통량12','출발지방향총교통량':'출발지방향총교통량12'})

mdf20 = tdf20.copy()
mdf20 = mdf20.rename(columns={'도착지방향총교통량':'도착지방향총교통량20','출발지방향총교통량':'출발지방향총교통량20'})

mdf24 = tdf24.copy()
mdf24 = mdf24.rename(columns={'도착지방향총교통량':'도착지방향총교통량24','출발지방향총교통량':'출발지방향총교통량24'})

In [ ]:
mdf = pd.merge(mdf04, mdf12, how='outer')
mdf = pd.merge(mdf, mdf20, how='outer')
mdf = pd.merge(mdf, mdf24, how='outer')
mdf

In [ ]:
# add average columns
mdf['도착지방향평균'] = (mdf['도착지방향총교통량04'] + mdf['도착지방향총교통량12'] + mdf['도착지방향총교통량20'] + mdf['도착지방향총교통량24'])/4
mdf['출발지방향평균'] = (mdf['출발지방향총교통량04'] + mdf['출발지방향총교통량12'] + mdf['출발지방향총교통량20'] + mdf['출발지방향총교통량24'])/4
mdf

In [ ]:
# finally get the desired dataFrame
# fdf stands for final dataFrame
fdf = mdf[['출발영업소코드','도착영업소코드','출발영업소명','도착영업소명','도착지방향평균','출발지방향평균']]
fdf

In [ ]:
# save fdf for later use
fdf.to_csv('평일전국교통량평균.csv', index=False) 

In [ ]:
# edit out redundant columns
# fdf = fdf.drop(columns=['출발영업소코드','도착영업소코드','출발지방향평균']) excecuted
fdf.head()

In [ ]:
# import data concerning location of stations - sdf
sdf = pd.read_csv('sdf.csv',encoding='utf-8')
sdf.head()

In [ ]:
# join the two dfs, fdf and sdf
# first to add address columns for start-stations
# mdf stands for merged dataFrame
mdf = pd.merge(fdf, sdf, left_on="출발영업소명", right_on="새영업소명").drop('새영업소명', axis=1)
mdf = pd.merge(mdf, sdf, left_on="도착영업소명", right_on="새영업소명").drop('새영업소명', axis=1)
mdf.head()

In [ ]:
# drop redundant columns
mdf = mdf.drop(columns=['노선명_x','노선명_y'])
mdf.head()

In [ ]:
# rename the added columns
mdf = mdf.rename(columns={'세부주소1_x':'출발주소1', '세부주소2_x':'출발주소2',
                         '세부주소1_y':'도착주소1', '세부주소2_y':'도착주소2'})
mdf.head()

In [ ]:
# change the order of columns
mdf = mdf.iloc[:,[0,3,4,1,5,6,2]]
mdf.head()

In [21]:
# for the network over all provinces, take some parts of mdf
# the first partial mdf
pmdf = mdf[['출발주소1','도착주소1','도착지방향평균']]
pmdf.to_csv('전국교통량평균.csv', index=False, encoding='utf-8') 

In [3]:
import pandas as pd
pmdf = pd.read_csv('전국교통량평균.csv')
pmdf.head()

,출발주소1,도착주소1,도착지방향평균
0,경기도,경기도,65.25
1,경기도,경기도,4058.00
2,경기도,경기도,18545.75
3,경기도,경상남도,14.25
4,경기도,경기도,9914.75


In [13]:
# adjMatrix associated with pmdf
import numpy as np
nList = np.concatenate((pmdf['출발주소1'].values, pmdf['도착주소1'].values), axis=None)
nList = np.unique(nList)

In [33]:
# create an empty matrix to fill up
import numpy as np
adjMatrix = pd.DataFrame(np.zeros((len(nList),len(nList))), columns=nList, index=nList)
adjMatrix.head()

,강원도,경기도,경상남도,경상북도,광주광역시,대구광역시,대전광역시,부산광역시,세종특별자치시,예정,울산광역시,전라남도,전라북도,충청남도,충청북도
강원도,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
경기도,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
경상남도,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
경상북도,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
광주광역시,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
# while iterating over the rows, fill up the elements
for index, row in pmdf.iterrows():
    adjMatrix[row['출발주소1']][row['도착주소1']] += row['도착지방향평균']
    adjMatrix[row['도착주소1']][row['출발주소1']] += row['도착지방향평균']
adjMatrix.head()

C:\Users\syj43\AppData\Local\Temp\ipykernel_3848\1788532381.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  adjMatrix[row['출발주소1']][row['도착주소1']] += row['도착지방향평균']
C:\Users\syj43\AppData\Local\Temp\ipykernel_3848\1788532381.py:4: FutureW

,강원도,경기도,경상남도,경상북도,광주광역시,대구광역시,대전광역시,부산광역시,세종특별자치시,예정,울산광역시,전라남도,전라북도,충청남도,충청북도
강원도,220415.50,67651.00,566.75,2509.5,76.50,885.50,1249.50,23.50,18.25,0.75,8.50,425.5,490.50,402.75,13550.00
경기도,67651.00,1964944.50,14335.75,19711.0,713.75,12768.25,25629.75,41025.00,187.00,1535.75,326.75,7202.0,6084.50,161922.25,115944.00
경상남도,566.75,14335.75,675012.00,32563.5,3490.75,48042.25,4270.50,168700.75,653.25,4.50,41742.50,17973.0,8643.50,5682.75,4724.00
경상북도,2509.50,19711.00,32563.50,193738.5,1563.00,218805.00,5781.75,10522.75,660.00,7.50,19155.25,2426.5,3277.75,6244.50,17046.75
광주광역시,76.50,713.75,3490.75,1563.0,21752.00,1443.00,678.75,687.75,88.75,0.25,116.75,77567.0,9804.75,1956.00,587.50


In [35]:
# clear self-loops
for i in range(len(nList)):
    adjMatrix.iloc[i][i] = 0
adjMatrix.head()

C:\Users\syj43\AppData\Local\Temp\ipykernel_3848\3927112138.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  adjMatrix.iloc[i][i] = 0
C:\Users\syj43\AppData\Local\Temp\ipykernel_3848\3927112138.py:3: FutureWarning: Series.__setitem__ trea

,강원도,경기도,경상남도,경상북도,광주광역시,대구광역시,대전광역시,부산광역시,세종특별자치시,예정,울산광역시,전라남도,전라북도,충청남도,충청북도
강원도,0.00,67651.00,566.75,2509.5,76.50,885.50,1249.50,23.50,18.25,0.75,8.50,425.5,490.50,402.75,13550.00
경기도,67651.00,0.00,14335.75,19711.0,713.75,12768.25,25629.75,41025.00,187.00,1535.75,326.75,7202.0,6084.50,161922.25,115944.00
경상남도,566.75,14335.75,0.00,32563.5,3490.75,48042.25,4270.50,168700.75,653.25,4.50,41742.50,17973.0,8643.50,5682.75,4724.00
경상북도,2509.50,19711.00,32563.50,0.0,1563.00,218805.00,5781.75,10522.75,660.00,7.50,19155.25,2426.5,3277.75,6244.50,17046.75
광주광역시,76.50,713.75,3490.75,1563.0,0.00,1443.00,678.75,687.75,88.75,0.25,116.75,77567.0,9804.75,1956.00,587.50


In [36]:
adjMatrix = adjMatrix.drop(['예정'], axis=0)
adjMatrix = adjMatrix.drop(['예정'], axis=1)

In [41]:
# total traffic
totalTraff = adjMatrix.sum(axis='columns')
totalTraff = totalTraff.apply(lambda x:int(x))

In [37]:
# save the adjMatrix in csv format for later use
adjMatrix.to_csv('평일전국교통량행렬.csv', index=True, encoding='utf-8') 

In [42]:
totalTraff.to_csv('평일전국총교통량.csv')

In [ ]:
# end of code